In [122]:
# Requirements: Python 2.7

In [123]:
import numpy as np
import pandas as pd
from scipy.stats import spearmanr,pearsonr
from math import log10
from sklearn.linear_model import LogisticRegression
from csv import writer 
np.random.seed(0)
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import sklearn
import math

# proteins that are significantly correlated with SIS3 (p < 0.05) 

In [124]:
sig_proteins = ['OID01312', 'OID01485', 'OID01008', 'OID01483', 'OID01480', 'OID01003', 'OID01399', 'OID05176', 'OID00952', 'OID00950', 'OID01390', 'OID01397', 'OID00957', 'OID00954', 'OID00955', 'OID01041', 'OID00709', 'OID00706', 'OID05442', 'OID01090', 'OID00382', 'OID01092', 'OID01093', 'OID05214', 'OID05455', 'OID00389', 'OID00388', 'OID00426', 'OID05451', 'OID00424', 'OID01327', 'OID01325', 'OID00324', 'OID01039', 'OID00443', 'OID01033', 'OID00945', 'OID05168', 'OID00940', 'OID00942', 'OID01457', 'OID01406', 'OID01404', 'OID00958', 'OID01082', 'OID05207', 'OID00630', 'OID05201', 'OID01086', 'OID01085', 'OID05448', 'OID01152', 'OID05443', 'OID05208', 'OID01021', 'OID00459', 'OID00692', 'OID00694', 'OID01026', 'OID00939', 'OID01439', 'OID00936', 'OID05473', 'OID05470', 'OID00605', 'OID05476', 'OID01011', 'OID00439', 'OID05141', 'OID00608', 'OID01123', 'OID05128', 'OID05129', 'OID01056', 'OID00467', 'OID01059', 'OID01425', 'OID00591', 'OID01428', 'OID01154', 'OID05465', 'OID05463', 'OID05157', 'OID05156', 'OID01133', 'OID05150', 'OID01131', 'OID01336', 'OID01046', 'OID01044', 'OID05406', 'OID05405', 'OID01191', 'OID01048', 'OID00994', 'OID00990', 'OID05483', 'OID05482', 'OID00668', 'OID05213', 'OID00662', 'OID01102', 'OID01101', 'OID01349', 'OID00745', 'OID00744', 'OID01345', 'OID01108', 'OID01109', 'OID01340', 'OID01342', 'OID01107', 'OID05437', 'OID01162', 'OID01163', 'OID00401', 'OID00981', 'OID00983', 'OID01105', 'OID00565', 'OID05218', 'OID00380', 'OID05219', 'OID00679', 'OID05450', 'OID00592', 'OID00671', 'OID00378', 'OID01446', 'OID01113', 'OID01357', 'OID01356', 'OID01354', 'OID01352', 'OID00374', 'OID01350', 'OID00417', 'OID01309', 'OID01177', 'OID01176', 'OID05434', 'OID00419', 'OID01467', 'OID01110', 'OID00371', 'OID00974', 'OID00976', 'OID00391', 'OID00648', 'OID01362', 'OID01360', 'OID01472', 'OID01473', 'OID05181', 'OID01200', 'OID01070', 'OID01202', 'OID01076', 'OID01077', 'OID01146', 'OID00318', 'OID00982', 'OID01143', 'OID01140', 'OID01453', 'OID00307', 'OID00685', 'OID01384', 'OID00655', 'OID05153', 'OID00715', 'OID05192', 'OID00712', 'OID05428', 'OID00392', 'OID01068', 'OID01065', 'OID05423', 'OID05222', 'OID05220']
print(len(sig_proteins), sig_proteins)

(180, ['OID01312', 'OID01485', 'OID01008', 'OID01483', 'OID01480', 'OID01003', 'OID01399', 'OID05176', 'OID00952', 'OID00950', 'OID01390', 'OID01397', 'OID00957', 'OID00954', 'OID00955', 'OID01041', 'OID00709', 'OID00706', 'OID05442', 'OID01090', 'OID00382', 'OID01092', 'OID01093', 'OID05214', 'OID05455', 'OID00389', 'OID00388', 'OID00426', 'OID05451', 'OID00424', 'OID01327', 'OID01325', 'OID00324', 'OID01039', 'OID00443', 'OID01033', 'OID00945', 'OID05168', 'OID00940', 'OID00942', 'OID01457', 'OID01406', 'OID01404', 'OID00958', 'OID01082', 'OID05207', 'OID00630', 'OID05201', 'OID01086', 'OID01085', 'OID05448', 'OID01152', 'OID05443', 'OID05208', 'OID01021', 'OID00459', 'OID00692', 'OID00694', 'OID01026', 'OID00939', 'OID01439', 'OID00936', 'OID05473', 'OID05470', 'OID00605', 'OID05476', 'OID01011', 'OID00439', 'OID05141', 'OID00608', 'OID01123', 'OID05128', 'OID05129', 'OID01056', 'OID00467', 'OID01059', 'OID01425', 'OID00591', 'OID01428', 'OID01154', 'OID05465', 'OID05463', 'OID05157

In [125]:
# this function adds significant proteins (as listed above) to clinical features in feat2 and returns their union
def fs_func(data, clinical_features):
    sigs = [f for f in sig_proteins if f in data] # siginificant proteins present in cleaned data (after removing missinig values)
    union = clinical_features
    union.extend(sigs) # 
    return union  

# Read data

In [237]:
columns=['model', 'features', 'target',  'spearman_stat', 'spearman_pvalue',  'log10_spearman_pvalue', 'pearson_stat', 'pearson_pvalue',  'log10_pearson_pvalue', 'Adj_r2', 'acc', 'rmse']
results = pd.DataFrame(columns = columns)
NPX_data_V2 = pd.read_csv('C_NPX_data.csv')
NPX_data_V2 = NPX_data_V2.dropna(axis=1)
patient_data_v2 = pd.read_csv('Subject_Clinical_Data.csv') #('C_patient_data.csv')
len(patient_data_v2)

85

# Modeling

In [238]:
target = ['SIS3'] 
clinical_features = ['Age','Time Since Stroke']#, 
fs_method = 'sig' #'all'#  
str_add_fea = fs_method + ' proteins + ' + ' + '.join(clinical_features)

y_data_features = []
y_data_features.extend(clinical_features)
y_data_features.extend(target)
y_data_features.extend(['PID'])
y_data = patient_data_v2.dropna(subset=target)
y_data = y_data[y_data_features] #'size','timesince','age',
y_data = y_data.dropna(subset=clinical_features)

X = NPX_data_V2

data = y_data.merge(X,how='inner', on='PID')

selected_features = [col for col in NPX_data_V2 if 'OID' in col]
selected_features.extend(clinical_features)
pids = set(data['PID'].values.tolist())   
len(pids)

85

In [239]:
tru_vals = []
predictions = []
thr = 0.3
x_shape = len(selected_features)
lens = []
coefs = []
inters = []
for i,p in enumerate(list(pids)):
    model = LogisticRegression(fit_intercept=True, random_state=42)
    if fs_method == 'all':
        new_selected_features = selected_features
    else:
        new_selected_features = fs_func(data, clinical_features)
    x_tr = data[data['PID']!=p][selected_features].values.reshape(-1, len(selected_features))
    y_tr = data[data['PID']!=p][target].values.reshape(-1, 1)
    data2 = data.copy()
    data2[[c for c in data if ((c not in new_selected_features)) and ( c != 'PID') and (c != target[0])]]=0
    x_te = data2[data2['PID']==p][selected_features].values.reshape(-1, len(selected_features))
    y_te = data2[data2['PID']==p][target].values.reshape(-1, 1)
    tru_vals.extend(y_te.flatten())
    model.fit(x_tr, y_tr)
    y_pred = model.predict(x_te)
    predictions.extend(y_pred)


# RMSE

In [242]:
mse = sklearn.metrics.mean_squared_error(tru_vals, predictions) # MSE
rmse = math.sqrt(mse) # RMSE

# Pearson

In [243]:
pearson_stat = pearsonr(predictions, tru_vals)[0]
pearson_pval = pearsonr(predictions, tru_vals)[1]
pearson_pval_log = -log10(pearsonr(predictions, tru_vals)[1])

# Spearman

In [244]:
spearman_stat = spearmanr(predictions, tru_vals)[0]
spearman_pval = spearmanr(predictions, tru_vals)[1]
spearman_pval_log = -log10(spearmanr(predictions, tru_vals)[1])

# Saving results

In [235]:
List = [type(model).__name__, str_add_fea, target, spearman_stat, spearman_pval, spearman_pval_log, pearson_stat, pearson_pval, pearson_pval_log, Adj_r2, acc, rmse]
results.loc[len(results)] = List
file_name = '%s.txt'%target[0]
results.to_csv(file_name, index = False)

In [236]:
file_name1 = '%s_results.txt'%(str_add_fea)
c = [tru_vals, predictions] 
with open(file_name1, "w") as file:
    for x in zip(*c):
        file.write("{0}\t{1}\n".format(*x))